In [56]:
import requests
import validators as vld
import pandas as pd
import re
import urllib
import logging

In [57]:
df_train = pd.read_csv('./train.csv', index_col='id')
df_test = pd.read_csv('./test.csv', index_col='id')

In [58]:
urls = set()

def get_all_urls(source):
    for _, text in source['text'].items():
        tokens = re.split('\\s+', text)
        url_tokens = list(filter(vld.url, tokens))
        for url in url_tokens:
            urls.add(url)

get_all_urls(df_train)
get_all_urls(df_test)
print(len(urls))

6522


In [59]:
import urllib3
urllib3.disable_warnings()

follow_domains = ['//t.co/', '//bit.ly/', '//dlvr.it/', 
                  '//tinyurl.com/', '//goo.gl/', '//ln.is/', '//shrd.by/', 
                  '//ift.tt/' , '//fb.me/',
                  '//on.doi.gov/', '//ow.ly/']

def follow_link(url, level=1):
    if level > 3:
        return [url]
    try:
        response = requests.get(url.replace('http://', 'https://'), verify=False, allow_redirects=False)
    except:
        return [url]
    if 'Location' in response.headers:
        url = response.headers['Location']
        for domain in follow_domains:
            if domain in url:
                out = follow_link(url, level+1)
                if not isinstance(out, (list, tuple)):
                    out = [out]
                elif isinstance(out, tuple):
                    out = list(out)
                if not url in out:
                    out.append(url)
                return out
        return [url]
    else:
        return [url]


In [60]:

result_url_expanded = []
for url in urls:
    output = follow_link(url)
    domain = ''
    last_url = None
    count = 1
    if output and isinstance(output, list):
        count = len(output)
        last_url = output[0]
    else:
        count = 1
        last_url = url
    domain = urllib.parse.urlparse(last_url).netloc
    result_url_expanded.append((url,domain,output,count))


In [61]:
df_url_domains = pd.DataFrame(data=result_url_expanded, columns=['url', 'final_url_domain', 'all_urls', 'url_redirects'])


In [68]:
df_url_domains.sample(n=20)

,url,final_url_domain,all_urls,url_redirects
5593,https://t.co/qwHINBni8e,preppiesoftheapocalypse.blogspot.com,[https://preppiesoftheapocalypse.blogspot.com/2015/08/travelogue-seattle-tacoma-portland.html],1
4806,https://t.co/mXZ7yX8ld1,shrd.by,[https://shrd.by/DKe4QH],1
6363,https://t.co/KakY4mpCO4,newyork.24-7newz.com,"[http://newyork.24-7newz.com/one-man-fatally-shot-another-wounded-on-vermont-street-buffalo/, https://bit.ly/1DvLLd1]",2
1527,https://t.co/rM6CNzkSzL,bbc.in,"[http://bbc.in/1SUHlDD, https://fb.me/sEKaOjAY]",2
6475,https://t.co/DQhpm4A5qY,nowtrending.com.ng,[https://nowtrending.com.ng/index.php/2015/08/06/police-charge-traditional-ruler-others-with-informants-murder/],1
2489,https://t.co/U7ZWlk8THX,twitter.com,[https://twitter.com/calgarysun/status/629083064795271168/photo/1],1
9,https://t.co/Eu21N1G1ZX,businessinsurance.com,"[http://businessinsurance.com/article/20150806/NEWS08/150809914, https://ow.ly/32j7JY]",2
3899,https://t.co/n8VcxGyhQ8,twitter.com,[https://twitter.com/BurnabyBCrr/status/629080868468830208/photo/1],1
1041,https://t.co/bwe9kJCEPt,youtu.be,[https://youtu.be/ehJvAdOegFk?a],1
6515,https://t.co/oLdjsowKY5,news.google.com,"[http://news.google.com/news/url?sa=t&fd=R&ct2=us&usg=AFQjCNHQbgM4XlJjPJA2BYiaBJ6lQmYeCw&clid=c3a7d30bb8a4878e06b80cf16b898331&ei=iLPCVYjiFoLdmAKc87GQBg&url=http%3A%2F%2Fwww.chicagotribune.com%2Fnews%2Fct-toddler-epilepsy-drowning-hanover-park-met-20150805-story.html&utm_source=dlvr.it&utm_medium=twitter, https://dlvr.it/BlvSjp]",2


In [69]:
final_domain = []
for _, row in df_url_domains.iterrows():
    full_domain = row['final_url_domain']
    parts = full_domain.split('.')
    if len(parts) >= 2 and parts[-1] == 'uk' and parts[-2] == 'co':
        parts = parts[:-2] + ['co.uk']
    domain = '.'.join(parts[-2:])
    final_domain.append(domain)

In [74]:
final_domain[0:200:10]

['ind.pn',
 'eventbrite.com',
 'twitter.com',
 'instagram.com',
 'waze.com',
 'blogspot.ca',
 'huffingtonpost.com',
 'sillygif.com',
 'twitter.com',
 'ln.is',
 'blogspot.com',
 'feedburner.com',
 'ltl.is',
 'lat.ms',
 'cur.lv',
 'twitter.com',
 'laughingsquid.com',
 'buy2day.info',
 'freddiemac.com',
 'bbc.in']

In [75]:
df_url_domains['final_domain'] = final_domain

In [76]:
df_url_domains.to_csv('./url_stats.csv', index=False, columns=['url', 'final_url_domain', 'final_domain', 'url_redirects'])